# **Exploratory Data Analysis**
---

## **1) Core Objectives**
---

### Overview:

Our primary objective with this exploratory analysis is to identify the features in our data that best represent our understanding of the [root causes of flight delays](../Research/flight_delay_reasons.md) and in so doing, provide the basis for the **accurate** and **reliable** prediction of commercial flight delays. We aim to develop a sense for how well our feature space captures common flight delay reasons in order to determine which features to select, which additional data to gather, and which features should be re-engineered or excluded.

### [Suggested Considerations](https://github.com/lighthouse-labs/mid-term-project-I/blob/master/exploratory_analysis.ipynb):

- Test the hypothesis that the delay is from Normal distribution and that mean of the delay is 0. Be careful about the outliers.
- Is average/median monthly delay different during the year? If so, which months have the biggest delays and what could be the reason?
- Does the weather affect the delay?
- How are taxi times changing during the day? Does higher traffic lead to longer taxi times?
- What is the average percentage of delays that exist prior to departure (*i.e.* are arrival delays caused by departure delays)? Are airlines able to lower the delay during the flights?
- How many states cover 50% of US air traffic?
- Test the hypothesis that planes fly faster when there is a departure delay.
- When (which hour) do most 'LONG', 'SHORT', 'MEDIUM' haul flights take off?
- Find the top 10 the bussiest airports. Does the greatest number of flights mean that the majority of passengers went through a given airport? How much traffic do these 10 airports cover?
- Do bigger delays lead to bigger fuel consumption per passenger?